# Remarks

* Data normalization
    * Mobilenet expects data from -1 to 1
        * Normalize Input Data or Include in Model
        * TFLite Conversion must fit according to decision
    * Ground Truth Data: for better inspection Data multiplied by 80. Undo the change in the Data Input Pipeline
* Overview in Tutorials:
    * tf.function
* Idea

# Libraries

In [1]:
#!for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done

import tensorflow as tf
import numpy as np
import os
import time
from datetime import datetime
import sys
import matplotlib.pyplot as plt
import argparse
import yaml

import DataProcessing.dataset as dataset
import DataProcessing.model_files as model_files
import Nets.backbones as backbones
import Nets.features as features
import Nets.losses as losses
import Nets.metrics as metrics
import Nets.visualize as visualize
import Nets.tools as tools

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

#np.set_printoptions(threshold=sys.maxsize)

# Parser

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--model', type=str, required=False, default=None)
parser.add_argument('--data', type=str, required=False, default=None)

parser.add_argument('--bs', type=int, required=False, default=None)
parser.add_argument('--idx', type=int, required=False, default=None)
parser.add_argument('--epoch', type=int, required=False, default=None)
parser.add_argument('--noise', type=float, required=False, default=None)

parser.add_argument('--train_model', action='store_true', default=False)
parser.add_argument('--cache', action='store_true', default=False)
parser.add_argument('--save', action='store_true', default=False)
parser.add_argument('--sigmoid', action='store_true', default=False)
parser.add_argument('--focal', action='store_true', default=False)

parser.add_argument('--beta_upper', type=float, required=False, default=None)
parser.add_argument('--gamma', type=float, required=False, default=None)
parser.add_argument('--alpha', type=float, required=False, default=None)

file_name = None
try:
    file_name = __file__
except:
    print("Jupyter Notebook")
       
if file_name is None:
    args = parser.parse_args("")
    args.train_model = True
    args.cache = True
    #args.save = True
    args.save = True
    args.sigmoid = False
    args.focal = True
else:    
    args = parser.parse_args()

Jupyter Notebook


# Options

In [3]:
# Generall Parameters
TRAIN_MODEL = True #args.train_model
SEED = None

# LOSS
weighted_multi_label_sigmoid_edge_loss = args.sigmoid
# focal_loss = args.focal
focal_loss = True

beta_upper = 0.5 if args.beta_upper is None else args.beta_upper
beta_lower = 1.0 - beta_upper
gamma = 2.0 if args.gamma is None else args.gamma
alpha = 2.0 if args.alpha is None else args.alpha
class_weighted = True
weighted_beta = True

config_path = os.path.join(os.getcwd(), 'configs')
model_config_path = os.path.join(config_path, 'model.yaml')

with open(model_config_path, 'r') as file:
    model_cfg = yaml.safe_load(file)

# Load Dataset, Preprocess Images and Dataset

In [4]:
tf.random.set_seed(SEED)

DP = dataset.DataProcessing(model_cfg["INPUT_SHAPE"], model_cfg["OUTPUT_SHAPE"], config_path)
DP.path_definitions()

MF = model_files.ModelFiles()
MF.path_definitions(model_cfg["NAME"], DP.ds_cfg["NAME"], make_dirs=True)
MF.clean_model_directories(model_cfg["CALLBACKS"]["DEL_OLD_CKPT"], model_cfg["CALLBACKS"]["DEL_OLD_TB"])

if TRAIN_MODEL:
    rng = tf.random.Generator.from_seed(123, alg='philox')

    train_ds, img_count_train = DP.load_dataset(DP.key.train)
    train_ds = DP.dataset_processing(train_ds, DP.key.train, shuffle=True, prefetch=True, img_count=img_count_train,
                                     rng=rng)

test_ds, img_count_test = DP.load_dataset(DP.key.test)
test_ds = DP.dataset_processing(test_ds, DP.key.test, shuffle=False, prefetch=True, img_count=img_count_test)


2022-11-02 12:59:40.754894: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-02 12:59:40.754943: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (david-ThinkPad-X1-Yoga-Gen-6): /proc/driver/nvidia/version does not exist
2022-11-02 12:59:40.755924: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
TRAIN 1
The TRAIN DS contains 202 images.
0
1
TEST 2
The TEST DS contains 2 images.


# Model

In [5]:
if TRAIN_MODEL:
    output_dims = model_cfg["OUTPUT_SHAPE"]

    # BACKBONE
    backbone, output_names = backbones.get_backbone(name=model_cfg["BACKBONE"]["NAME"],
                                                    weights=model_cfg["BACKBONE"]["WEIGHTS"],
                                                    height=model_cfg["INPUT_SHAPE"][0],
                                                    width=model_cfg["INPUT_SHAPE"][1],
                                                    alpha=model_cfg["BACKBONE"]["ALPHA"],
                                                    output_layer=model_cfg["BACKBONE"]["OUTPUT_IDS"],
                                                    trainable_idx=model_cfg["BACKBONE"]["TRAIN_IDX"])
    
    # prior edge map input
    ann_input = tf.keras.Input(shape=(output_dims[0], output_dims[1],1), name='input_PRIOR_ANN')
    x = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same",
                      strides=1, use_bias=False)(ann_input)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    
    x = tf.keras.layers.DepthwiseConv2D(kernel_size=5, dilation_rate=1, padding="same",
                                        strides=2, use_bias=False)(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x_out = tf.keras.layers.ReLU(name="x_out")(x)
    
    # backbone output
    b = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                               strides=1, use_bias=False)(backbone.output[-1])
    b = tf.keras.layers.BatchNormalization()(b)
    b = tf.keras.layers.ReLU()(b)
    
    # Mix layers
    #concat:
    c = tf.keras.layers.Concatenate(axis=-1)([x_out,b])
    c = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same",
                                    strides=1, use_bias=False)(c)
    c = tf.keras.layers.BatchNormalization()(c)
    c = tf.keras.layers.ReLU()(c)
    c = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same",
                                strides=1, use_bias=False)(c)
    c = tf.keras.layers.BatchNormalization()(c)
    c = tf.keras.layers.ReLU()(c)
    
    # mult. layers:
    mu = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same", 
                           strides=1, use_bias=False)(backbone.output[-1])
    mu = tf.keras.layers.BatchNormalization()(mu)
    mu = tf.keras.layers.ReLU()(mu)
    
    mu = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                       strides=1, use_bias=False)(mu)
    mu = tf.keras.layers.BatchNormalization()(mu)
    mu = tf.keras.layers.ReLU(name="out_mu")(mu)
    
    out = []
    for i in range(9):
        out.append(mu*c[:,:,:,i:i+1])
    x = tf.keras.layers.Concatenate(axis=-1)(out)
    
    # Post Processing
    x = tf.keras.layers.DepthwiseConv2D(kernel_size=3, dilation_rate=1, padding="same",
                                        strides=1, use_bias=False)(x)
    
    x = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same",
                               strides=1, use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x_resize = tf.image.resize(x, (output_dims[0], output_dims[1]))
    x_edge = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same",
                  strides=1, use_bias=True)(x_resize)
    x_edge = tf.keras.layers.BatchNormalization()(x_edge)

    x_edge = tf.keras.layers.ReLU()(x_edge)

    x_edge = tf.keras.layers.Conv2D(1, kernel_size=3, dilation_rate=1, padding="same",
                                    strides=1, use_bias=True)(x_edge)
    edge_map = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same",
                                strides=1, use_bias=True)(backbone.output[1])

    edge_map = tf.keras.layers.BatchNormalization()(edge_map)

    edge_map = tf.keras.layers.ReLU()(edge_map)

    edge_map = tf.keras.layers.Conv2D(1, kernel_size=3, dilation_rate=1, padding="same",
                                strides=1, use_bias=True)(edge_map)

    x_edge = tf.keras.layers.Concatenate(axis=-1)([x_edge, edge_map])

    output_edge = tf.keras.layers.Conv2D(1, kernel_size=3, dilation_rate=1, padding="same",
                                        strides=1, use_bias=True, name="output_ANN")(x_edge)
    
    comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                                 strides=1, use_bias=True)(x)
    comp = tf.keras.layers.BatchNormalization()(comp)
    
    edge_comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                                 strides=1, use_bias=True)(ann_input)
    
    edge_comp = tf.keras.layers.BatchNormalization()(edge_comp)
    edge_comp = tf.keras.layers.ReLU()(edge_comp)
    
    edge_comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                                       strides=1, use_bias=True)(edge_comp)
    edge_comp = tf.keras.layers.BatchNormalization()(edge_comp)
    edge_comp = tf.keras.layers.ReLU()(edge_comp)
    
    edge_comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                                   strides=1, use_bias=True)(edge_comp)
    edge_comp = tf.keras.layers.BatchNormalization()(edge_comp)
    edge_comp = tf.keras.layers.ReLU()(edge_comp)
    
    edge_comp = tf.keras.layers.MaxPool2D(pool_size=(3, 3),strides=2,padding='same')(edge_comp)
    
    comp = tf.keras.layers.Concatenate(axis=-1)([comp, edge_comp])
    comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                               strides=1, use_bias=True)(comp)
    comp = tf.keras.layers.BatchNormalization()(comp)
    comp = tf.keras.layers.ReLU()(comp)
    
    comp = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same", 
                           strides=1, use_bias=True)(comp)
    comp = tf.keras.layers.BatchNormalization()(comp)
    comp_sep = tf.keras.layers.ReLU()(comp)
    
    comp = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same", 
                       strides=1, use_bias=True)(comp_sep)
    comp = tf.keras.layers.BatchNormalization()(comp)
    comp = tf.keras.layers.ReLU()(comp)
    
    comp = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same", 
                   strides=1, use_bias=True)(comp)
    comp = tf.keras.layers.BatchNormalization()(comp)
    comp = tf.keras.layers.ReLU()(comp)
    
    comp = tf.keras.layers.Concatenate(axis=-1)([comp, comp_sep])
    
    comp = tf.keras.layers.Conv2D(2, kernel_size=3, dilation_rate=1, padding="same", name="output_VERT_MAP")(comp)


    model = tf.keras.Model(inputs=(backbone.input, ann_input), outputs=[comp, output_edge])

    model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_IMG (InputLayer)         [(None, 640, 360, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_PRIOR_ANN (InputLayer)   [(None, 160, 90, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 640, 360, 3)  0           ['input_IMG[0][0]']              
                                                                                              

 icingOpLambda)                                                                                   
                                                                                                  
 tf.__operators__.getitem_3 (Sl  (None, 80, 45, 1)   0           ['re_lu_3[0][0]']                
 icingOpLambda)                                                                                   
                                                                                                  
 tf.__operators__.getitem_4 (Sl  (None, 80, 45, 1)   0           ['re_lu_3[0][0]']                
 icingOpLambda)                                                                                   
                                                                                                  
 tf.__operators__.getitem_5 (Sl  (None, 80, 45, 1)   0           ['re_lu_3[0][0]']                
 icingOpLambda)                                                                                   
          

 batch_normalization_7 (BatchNo  (None, 80, 45, 16)  64          ['conv2d_6[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 conv2d_14 (Conv2D)             (None, 160, 90, 9)   738         ['re_lu_9[0][0]']                
                                                                                                  
 re_lu_5 (ReLU)                 (None, 80, 45, 16)   0           ['batch_normalization_7[0][0]']  
                                                                                                  
 batch_normalization_13 (BatchN  (None, 160, 90, 9)  36          ['conv2d_14[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_11

# Compile and Train Model

In [8]:
if TRAIN_MODEL:
    # learning rate schedule
    base_learning_rate = 0.0015*0.1
    end_learning_rate = 0.0005*0.1
    decay_step = np.ceil(img_count_train / DP.ds_cfg[DP.key.train]["BATCH_SIZE"]) * model_cfg["EPOCHS"]
    lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(base_learning_rate, decay_steps=decay_step,
                                                                end_learning_rate=end_learning_rate, power=0.9)
    
    frequency = int(
        np.ceil(img_count_train / DP.ds_cfg[DP.key.train]["BATCH_SIZE"]) * model_cfg["CALLBACKS"]["CKPT_FREQ"]) + 1

    logdir = os.path.join(MF.paths['TBLOGS'], datetime.now().strftime("%Y%m%d-%H%M%S"))
    callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath=MF.paths["CKPT"] + "/ckpt-loss={val_loss:.2f}-epoch={epoch:.2f}",
        save_weights_only=False, save_best_only=False, monitor="val_f1", verbose=1, save_freq='epoch', period=frequency),
        tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)]

    # compile model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  loss={'output_ANN': losses.focal_loss_binary,
                        'output_VERT_MAP': losses.optical_flow_loss},
                  loss_weights={"output_VERT_MAP": 1.0, "output_ANN": 0.001})
                  #metrics={'output_CURRENT_VERT': [metrics.BinaryAccuracyEdges(threshold_prediction=0),
                  #         metrics.F1Edges(threshold_prediction=0, threshold_edge_width=0)]})

    history = model.fit(train_ds, epochs=model_cfg["EPOCHS"], validation_data=test_ds, callbacks=callbacks)


Epoch 1/120
201/201 [==============================] - 26s 117ms/step - loss: 46.4495 - output_VERT_MAP_loss: 46.3197 - output_ANN_loss: 129.7484 - val_loss: 143.2616 - val_output_VERT_MAP_loss: 143.0425 - val_output_ANN_loss: 219.0947
Epoch 2/120
201/201 [==============================] - 27s 132ms/step - loss: 43.6205 - output_VERT_MAP_loss: 43.4906 - output_ANN_loss: 129.8857 - val_loss: 147.2022 - val_output_VERT_MAP_loss: 146.9726 - val_output_ANN_loss: 229.6655
Epoch 3/120
201/201 [==============================] - 27s 132ms/step - loss: 40.8416 - output_VERT_MAP_loss: 40.7152 - output_ANN_loss: 126.4077 - val_loss: 136.0210 - val_output_VERT_MAP_loss: 135.8336 - val_output_ANN_loss: 187.4046
Epoch 4/120
201/201 [==============================] - 27s 133ms/step - loss: 39.4138 - output_VERT_MAP_loss: 39.2863 - output_ANN_loss: 127.5168 - val_loss: 134.1736 - val_output_VERT_MAP_loss: 134.0113 - val_output_ANN_loss: 162.3262
Epoch 5/120
201/201 [==============================] - 4

201/201 [==============================] - 27s 135ms/step - loss: 18.9680 - output_VERT_MAP_loss: 18.8348 - output_ANN_loss: 133.1559 - val_loss: 89.3827 - val_output_VERT_MAP_loss: 89.2326 - val_output_ANN_loss: 150.0405
Epoch 36/120
201/201 [==============================] - 27s 136ms/step - loss: 18.4433 - output_VERT_MAP_loss: 18.3058 - output_ANN_loss: 137.4783 - val_loss: 89.3369 - val_output_VERT_MAP_loss: 89.1832 - val_output_ANN_loss: 153.6933
Epoch 37/120
201/201 [==============================] - 29s 141ms/step - loss: 17.4767 - output_VERT_MAP_loss: 17.3417 - output_ANN_loss: 134.9799 - val_loss: 87.0450 - val_output_VERT_MAP_loss: 86.8904 - val_output_ANN_loss: 154.5412
Epoch 38/120
201/201 [==============================] - 35s 175ms/step - loss: 17.5038 - output_VERT_MAP_loss: 17.3703 - output_ANN_loss: 133.4962 - val_loss: 79.8005 - val_output_VERT_MAP_loss: 79.6445 - val_output_ANN_loss: 156.0231
Epoch 39/120
201/201 [==============================] - 27s 133ms/step - 

KeyboardInterrupt: 

In [ ]:
model_ckpt = os.listdir(MF.paths['CKPT'])

f1_max = 0
for ckpt_name in model_ckpt:
    if float(ckpt_name[-4:]) > f1_max:
        f1_max = float(ckpt_name[-4:])
        model_path = MF.paths['CKPT'] + "/" + ckpt_name

        print(model_path)

custom_objects = {"BinaryAccuracyEdges": metrics.BinaryAccuracyEdges, "F1Edges": metrics.F1Edges}

model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

# Plot Results

In [ ]:
if TRAIN_MODEL:
    plot_losses = ["loss", "loss"]
    plot_metrics = ["accuracy_edges", "f1", "recall", "precision"]

    path = os.path.join(MF.paths["FIGURES"], "training.svg")

    visualize.plot_training_results(res=history.history, losses=plot_losses, metrics=plot_metrics,
                                    save=model_cfg["SAVE"], path=path)

In [ ]:
### Maximum F1 Score:
# TODO: FAILS right now: fix
# if not TRAIN_MODEL:
#     step_width = 0.05
#     threshold_range = [0.05, 0.95]
#     threshold_array = np.arange(threshold_range[0], threshold_range[1] + step_width, step_width)
#     threshold_array = np.array([0.025, 0.1, 0.2, 0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.975])
#
#     path_metrics_evaluation_plot = os.path.join(MF.paths["FIGURES"], "threshold_metrics_evaluation_test_ds.svg")
#     threshold_f1_max = visualize.plot_threshold_metrics_evaluation_class(model=model, ds=test_ds,
#                                                                          num_classes=DP.num_classes,
#                                                                          threshold_array=threshold_array,
#                                                                          threshold_edge_width=0, save=model_cfg["SAVE"],
#                                                                          path=path_metrics_evaluation_plot)

# Fine Tuning

# Evaluate on Test DS of Real Images

## Metrics Evaluation

## Visual Results

# Save Model

In [ ]:
if model_cfg["SAVE"]:
    model.save(MF.paths["MODEL"])
    
    custom_objects = {"BinaryAccuracyEdges": metrics.BinaryAccuracyEdges,
                      "F1Edges": metrics.F1Edges}
    
    model = tf.keras.models.load_model(MF.paths["MODEL"], custom_objects=custom_objects)